# BigQuery basics

[BigQuery](https://cloud.google.com/bigquery/docs/) is a petabyte-scale analytics data warehouse that you can use to run SQL queries over vast amounts of data in near realtime. This page shows you how to get started with the Google BigQuery API using the Python client library.

## Import the libraries used in this tutorial

In [49]:
from google.cloud import bigquery
import pandas as pd
pd.set_option('display.max_columns', 500)

## Initialize a client

To use the BigQuery Python client library, start by initializing a client. The BigQuery client is used to send and receive messages from the BigQuery API.

### Client project
The `bigquery.Client` object uses your default project. Alternatively, you can specify a project in the `Client` constructor. For more information about how the default project is determined, see the [google-auth documentation](https://google-auth.readthedocs.io/en/latest/reference/google.auth.html).


### Client location
Locations are required for certain BigQuery operations such as creating a dataset. If a location is provided to the client when it is initialized, it will be the default location for jobs, datasets, and tables.

Run the following to create a client with your default project:

In [21]:
client = bigquery.Client(location="EU")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: pacific-destiny-243512


To explicitly specify a project when constructing the client, set the `project` parameter:

In [22]:
# client = bigquery.Client(location="US", project="your-project-id")

## Run a query on a public dataset

The following example queries the BigQuery `usa_names` public dataset to find the 10 most popular names. `usa_names` is a Social Security Administration dataset that contains all names from Social Security card applications for births that occurred in the United States after 1879.

Use the [Client.query](https://googleapis.github.io/google-cloud-python/latest/bigquery/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client.query) method to run the query, and the [QueryJob.to_dataframe](https://googleapis.github.io/google-cloud-python/latest/bigquery/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe) method to return the results as a pandas [`DataFrame`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [23]:
query = """
    SELECT
  wx.date,
  wx.value/10.0 AS prcp
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2015` AS wx
WHERE
  id = 'UKM00003772'
  AND qflag IS NULL
  AND element = 'PRCP'
ORDER BY
  wx.date
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()
df

,date,prcp
0,2015-01-01,0.3
1,2015-01-02,1.0
2,2015-01-03,3.8
3,2015-01-04,7.1
4,2015-01-05,0.0
5,2015-01-06,0.3
6,2015-01-07,1.0
7,2015-01-08,1.0
8,2015-01-09,12.4
9,2015-01-10,0.0


In [45]:
query = """
SELECT
  *
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2015` AS wx
WHERE
  date > "2015-06-23"
ORDER BY
  wx.date
LIMIT
  10
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()
df

,id,date,element,value,mflag,qflag,sflag,time
0,USC00419367,2015-06-24,PRCP,0.0,None,None,7,1000
1,USC00058429,2015-06-24,SNOW,0.0,None,None,7,None
2,USC00502015,2015-06-24,SNWD,0.0,None,None,7,2000
3,USC00262296,2015-06-24,SNOW,0.0,None,None,7,None
4,USC00044303,2015-06-24,PRCP,0.0,None,None,7,1500
5,USC00218419,2015-06-24,PRCP,0.0,None,None,7,1400
6,USC00107346,2015-06-24,PRCP,0.0,None,None,7,2000
7,USC00396669,2015-06-24,SNOW,0.0,None,None,7,None
8,USC00203744,2015-06-24,SNOW,0.0,None,None,7,None
9,USC00309049,2015-06-24,SNWD,0.0,None,None,7,2000


## London Crime Data

In [27]:
query_crime = """
SELECT 
  *
FROM
  `bigquery-public-data.london_crime.crime_by_lsoa` AS lc
WHERE
  year = 2015
LIMIT 10

"""
query_job = client.query(
    query_crime,
    # Location must match that of the dataset(s) referenced in the query.
    location="EU",
)  # API request - starts the query
dfc = query_job.to_dataframe()
dfc

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000001,City of London,Violence Against the Person,Harassment,0,2015,2
1,E01000005,City of London,Violence Against the Person,Harassment,0,2015,7
2,E01032739,City of London,Violence Against the Person,Harassment,1,2015,5
3,E01000005,City of London,Violence Against the Person,Harassment,0,2015,8
4,E01032740,City of London,Violence Against the Person,Harassment,0,2015,10
5,E01000001,City of London,Violence Against the Person,Harassment,0,2015,8
6,E01000005,City of London,Theft and Handling,Other Theft,2,2015,11
7,E01000005,City of London,Theft and Handling,Other Theft,1,2015,6
8,E01000005,City of London,Theft and Handling,Other Theft,1,2015,5
9,E01000002,City of London,Theft and Handling,Other Theft,0,2015,9


#### Major Categories of Crime

In [29]:
query_crime = """
SELECT DISTINCT
  major_category
FROM
  `bigquery-public-data.london_crime.crime_by_lsoa` AS lc
WHERE
  year = 2015

"""
query_job = client.query(
    query_crime,
    # Location must match that of the dataset(s) referenced in the query.
    location="EU",
)  # API request - starts the query
dfc = query_job.to_dataframe()
dfc

,major_category
0,Violence Against the Person
1,Theft and Handling
2,Drugs
3,Other Notifiable Offences
4,Robbery
5,Criminal Damage
6,Burglary
7,Sexual Offences
8,Fraud or Forgery


### Number of crimes by Major Category of crime

In [43]:
query_crime = """
SELECT 
  major_category, SUM(value)
FROM
  `bigquery-public-data.london_crime.crime_by_lsoa` AS lc
WHERE
  year = 2015
GROUP BY
  major_category

"""
query_job = client.query(
    query_crime,
    # Location must match that of the dataset(s) referenced in the query.
    location="EU",
)  # API request - starts the query
dfc = query_job.to_dataframe()
dfc

,major_category,f0_
0,Violence Against the Person,218740
1,Theft and Handling,284022
2,Drugs,39785
3,Other Notifiable Offences,14229
4,Robbery,21383
5,Criminal Damage,62976
6,Burglary,70489
7,Sexual Offences,0
8,Fraud or Forgery,0


## London Brigade Fire Alarms

In [50]:
query_crime = """
SELECT 
  *
FROM
  `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls` AS fire
WHERE
  incident_group = 'Fire' AND 
  cal_year = 2017
LIMIT
  10


"""
query_job = client.query(
    query_crime,
    # Location must match that of the dataset(s) referenced in the query.
    location="EU",
)  # API request - starts the query
dfc = query_job.to_dataframe()
dfc

,incident_number,date_of_call,cal_year,time_of_call,hour_of_call,timestamp_of_call,incident_group,stop_code_description,special_service_type,property_category,property_type,address_qualifier,postcode_full,postcode_district,borough_code,borough_name,proper_case,ward_code,ward_name,ward_name_new,easting_m,northing_m,easting_rounded,northing_rounded,frs,incident_station_ground,first_pump_arriving_attendance_time,first_pump_arriving_deployed_from_station,second_pump_arriving_attendance_time,second_pump_arriving_deployed_from_station,num_stations_with_pumps_attending,num_pumps_attending
0,040741-04042017,2017-04-04,2017,20:44:06,20,2017-04-04 20:44:06+00:00,Fire,Late Call,None,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Correct incident location,None,E11,E09000026,REDBRIDGE,Redbridge,E05000513,SNARESBROOK,SNARESBROOK,NaN,NaN,540350,189350,London,Leytonstone,449,Leytonstone,NaN,None,1,1
1,005509-13012017,2017-01-13,2017,19:49:12,19,2017-01-13 19:49:12+00:00,Fire,Chimney Fire,None,Dwelling,House - single occupancy,Within same building,None,BR6,E09000006,BROMLEY,Bromley,E05000110,CHELSFIELD AND PRATTS BOTTOM,CHELSFIELD AND PRATTS BOTTOM,NaN,NaN,546350,163850,London,Orpington,331,Orpington,NaN,None,1,1
2,052246-26042017,2017-04-26,2017,19:57:45,19,2017-04-26 19:57:45+00:00,Fire,Chimney Fire,None,Dwelling,House - single occupancy,Correct incident location,None,CR0,E09000008,CROYDON,Croydon,E05000157,NEW ADDINGTON,NEW ADDINGTON,NaN,NaN,538450,162150,London,Addington,179,Addington,NaN,None,1,1
3,014805-04022017,2017-02-04,2017,18:57:47,18,2017-02-04 18:57:47+00:00,Fire,Chimney Fire,None,Dwelling,House - single occupancy,Correct incident location,None,N14,E09000010,ENFIELD,Enfield,E05000208,SOUTHGATE,SOUTHGATE,NaN,NaN,530350,194550,London,Southgate,264,Southgate,NaN,None,1,1
4,012336-29012017,2017-01-29,2017,12:48:43,12,2017-01-29 12:48:43+00:00,Fire,Chimney Fire,None,Dwelling,House - single occupancy,Correct incident location,None,SE10,E09000011,GREENWICH,Greenwich,E05000222,GREENWICH WEST,GREENWICH WEST,NaN,NaN,538450,177050,London,Greenwich,174,Greenwich,NaN,None,1,1
5,017339-10022017,2017-02-10,2017,20:49:49,20,2017-02-10 20:49:49+00:00,Fire,Chimney Fire,None,Non Residential,Pub/wine bar/bar,Correct incident location,N15 3DA,N15,E09000014,HARINGEY,Haringey,E05000282,WEST GREEN,WEST GREEN,531774.0,189383.0,531750,189350,London,Tottenham,327,Tottenham,335.0,Hornsey,2,2
6,013560-01022017,2017-02-01,2017,14:39:35,14,2017-02-01 14:39:35+00:00,Fire,Chimney Fire,None,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Correct incident location,None,RM3,E09000016,HAVERING,Havering,E05000314,HEATON,HEATON,NaN,NaN,553450,192650,London,Harold Hill,285,Harold Hill,449.0,Romford,2,2
7,001299-03012017,2017-01-03,2017,19:27:40,19,2017-01-03 19:27:40+00:00,Fire,Chimney Fire,None,Dwelling,House - single occupancy,Correct incident location,None,RM5,E09000016,HAVERING,Havering,E05000316,MAWNEYS,MAWNEYS,NaN,NaN,549750,191150,London,Romford,269,Romford,386.0,Hainault,2,2
8,005870-14012017,2017-01-14,2017,18:50:40,18,2017-01-14 18:50:40+00:00,Fire,Chimney Fire,None,Dwelling,House - single occupancy,Correct incident location,None,KT5,E09000021,KINGSTON UPON THAMES,Kingston Upon thames,E05000400,ALEXANDRA,ALEXANDRA,NaN,NaN,520450,166150,London,Surbiton,581,Surbiton,NaN,None,1,1
9,034166-21032017,2017-03-21,2017,19:57:54,19,2017-03-21 19:57:54+00:00,Fire,Chimney Fire,None,Non Residential,Pub/wine bar/bar,Correct incident location,SE19 1QH,SE19,E09000022,LAMBETH,Lambeth,E05000422,GIPSY HILL,GIPSY HILL,533326.0,171168.0,533350,171150,London,West Norwood,232,West Norwood,247.0,West Norwood,1,2


## How long does the fire brigade take to come to your home?

## Load data from a pandas DataFrame to a new table

In [ ]:
records = [
    {"title": "The Meaning of Life", "release_year": 1983},
    {"title": "Monty Python and the Holy Grail", "release_year": 1975},
    {"title": "Life of Brian", "release_year": 1979},
    {"title": "And Now for Something Completely Different", "release_year": 1971},
]

# Optionally set explicit indices.
# If indices are not specified, a column will be created for the default
# indices created by pandas.
index = ["Q24980", "Q25043", "Q24953", "Q16403"]
df = pandas.DataFrame(records, index=pandas.Index(index, name="wikidata_id"))

table_ref = dataset.table("monty_python")
job = client.load_table_from_dataframe(df, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

## Cleaning Up

The following code deletes the dataset created for this tutorial, including all tables in the dataset.

In [ ]:
# Retrieve the dataset from the API
dataset = client.get_dataset(client.dataset(dataset_id))

# Delete the dataset and its contents
client.delete_dataset(dataset, delete_contents=True)

print('Deleted dataset: {}'.format(dataset.path))